In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
housetrain=pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
housetest=pd.read_csv('test.csv')

In [ ]:
housetrain.shape

In [ ]:
housetest.shape
# Test Data do not have Dependent Variable 'SalePrice', Predict it using
# Trained model

In [ ]:
# Adding Dependent Varaible to Test Data with single value 'test' for 
# matching number of columns of train data & for Data subsetting in end
housetest['SalePrice']='test'

In [ ]:
# Concatening Both Train & Test dataframes for preprocessing
combinedf=pd.concat([housetrain,housetest],axis=0)
# axis=0 Row wise Concatenation

In [ ]:
combinedf.shape

In [ ]:
# Split Dataframe into Numeric data & Non Numericdata
# Preprocessing is different for Object Data & Numeric Data, hence split
objectcols=combinedf.select_dtypes(include=['object'])
numericcols=combinedf.select_dtypes(include=np.number)

In [ ]:
print(objectcols.shape)
print(numericcols.shape)
# Number of Object Columns is 44
# Number of Numeric Columns is 37

In [ ]:
# Check Missing Values in Object Columns
objectcols.isnull().sum().sort_values(ascending=False)

In [ ]:
objectcols.info()

In [ ]:
# As per Data Dictionary, NA means the following:
# PoolQC - NA - No Pool
# MiscFeature - No MiscFeature
# Alley - NA  - No Alley Access
# Fence - NA  - No Fence
# FireplaceQu - NA - No Fireplace

In [ ]:
# Imputing Missing Values with NotAvaialable using For Loop & Column list
notavail=['PoolQC','MiscFeature','Alley','Fence','FireplaceQu']
for col in notavail:
    objectcols[col]=objectcols[col].fillna('NotAvailable')

In [ ]:
# As per Data Dictionary, NA means the following:
#GarageCond- No Garage       
#GarageQual- No Garage     
#GarageFinish - No Garage   
#GarageType - No Garage     

In [ ]:
# Imputing Missing Values with NotAvaialable using For Loop & Column list
garagecols=['GarageCond','GarageQual','GarageFinish','GarageType']
for col in garagecols:
    objectcols[col]=objectcols[col].fillna('NotAvailable')

In [ ]:
# As per Data Dictionary NA means the following:
#BsmtCond - No Basement
#BsmtExposure - No Basement
#BsmtQual -- No Basement
#BsmtFinType2 - No Basement
#BsmtFinType1 - No Basement

In [ ]:
# Imputing Missing Values with NotAvaialable using For Loop & Column list
bsmtcols=['BsmtCond','BsmtExposure','BsmtQual','BsmtFinType2',
         'BsmtFinType1']
for col in bsmtcols:
    objectcols[col]=objectcols[col].fillna('NotAvailable')

In [ ]:
objectcols.MasVnrType.value_counts(dropna=False)

In [ ]:
# Imputing NA with Index of Maximum in Value Counts table
for col in objectcols:
    objectcols[col]=objectcols[col].fillna(
    objectcols[col].value_counts().idxmax())

In [ ]:
# Check Missing Values in Numeric Columns
numericcols.isnull().sum().sort_values(ascending=False)

In [ ]:
# Check for both Mean & Median (50% or Q2)
numericcols.describe()

In [ ]:
# Impute GarageYrBlt with missing code - 9999
numericcols.GarageYrBlt=numericcols.GarageYrBlt.fillna(9999)

In [ ]:
# Imputed all missing values in numeric columns with median of respective
#columns
for col in numericcols:
    numericcols[col]=numericcols[col].fillna(
    numericcols[col].median())

In [ ]:
# Identify Categorical Columns from Numeric Columns. Columns like year,
# rating, etc.
numericcols.columns

In [ ]:
# Seperated Cateorical Columns from Numeric Columns
categorycols=numericcols[['OverallQual','OverallCond', 'YearBuilt', 
                          'YearRemodAdd','GarageYrBlt','MoSold', 
                          'YrSold']]

In [ ]:
# Immediately dropped the above columns from numeric columns to prevent
# duplication
numericcols=numericcols.drop(['OverallQual','OverallCond', 'YearBuilt', 
                          'YearRemodAdd','GarageYrBlt','MoSold', 
                          'YrSold'],axis=1)

In [ ]:
print(objectcols.shape)
print(numericcols.shape)
print(categorycols.shape)
# Number of Object Columns - 44
# Number of Numeric Columns - 30
# Number of Categorical Columns - 7

In [ ]:
# Added SalePrice Column to Numerical Data which was in Object Data
numericcols['SalePrice']=objectcols.SalePrice

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
# Drop SalePrice Column from Objectcols as it is added to Numeric Cols
objectcols=objectcols.drop('SalePrice',axis=1)

In [ ]:
# Label Encode or Dummy Varible Encoding of Object Cols
objectcolsdummy=objectcols.apply(le.fit_transform)

In [ ]:
# Label Encode or Dummy Varible Encoding of Category Cols
categorycolsdummy=categorycols.apply(le.fit_transform)

In [ ]:
# Concatinating all 3 dataframes - numericcols, objectcolsdummy & 
# categorycolsdummy into one Dataframe
combinedfclean=pd.concat([numericcols,objectcolsdummy,
                         categorycolsdummy],axis=1)

In [ ]:
# Split data back to train & test from preprocessed combined dataframe
housetraindf=combinedfclean[combinedfclean.SalePrice!='test']
housetestdf=combinedfclean[combinedfclean.SalePrice=='test']

In [ ]:
# Dropping SalePrice Column from test data
housetestdf=housetestdf.drop('SalePrice',axis=1)

In [ ]:
# Converting SalePrice back to Numeric
housetraindf.SalePrice=housetraindf.SalePrice.astype('float64')

In [ ]:
# Test Null Average SalePrice of different OverallCond is equal
housetraindf.SalePrice.groupby(housetraindf.OverallCond).mean()

In [ ]:
cond0=housetraindf[housetraindf.OverallCond==0]
cond1=housetraindf[housetraindf.OverallCond==1]
cond2=housetraindf[housetraindf.OverallCond==2]
cond3=housetraindf[housetraindf.OverallCond==3]
cond4=housetraindf[housetraindf.OverallCond==4]
cond5=housetraindf[housetraindf.OverallCond==5]
cond6=housetraindf[housetraindf.OverallCond==6]
cond7=housetraindf[housetraindf.OverallCond==7]
cond8=housetraindf[housetraindf.OverallCond==8]

In [ ]:
from scipy.stats import f_oneway

In [ ]:
f_oneway(cond0.SalePrice,cond1.SalePrice,cond2.SalePrice,cond3.SalePrice,
        cond4.SalePrice,cond5.SalePrice,cond6.SalePrice,cond7.SalePrice,
        cond8.SalePrice)
# SINCE pvalue=7.962288819950615e-38 IS LESS THAN 0.05, REJECT NULL
# NULL - THERE IS NO SIGNFICANT DIFFERENCE IN AVERAGE SALEPRICE OF 
# DIFFERENT OVERALL CONDITION HOUSES

In [ ]:
# Test Null Average SalePrice of different MasVnrType is equal
housetraindf.SalePrice.groupby(housetraindf.MasVnrType).mean()

In [ ]:
# Split Data into DependentVariable(y) & Independent Variables (Xs)
y=housetraindf.SalePrice
X=housetraindf.drop(['SalePrice','Id'],axis=1)

In [ ]:
housetestdf=housetestdf.drop('Id',axis=1)

In [ ]:
# Check Distrbution and Outliers of SalePrice
y.plot(kind='box',vert=False)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linreg=LinearRegression()

In [ ]:
linregmodel=linreg.fit(X,y)

In [ ]:
linregmodel.score(X,y) # R Square

In [ ]:
regpredict=linregmodel.predict(housetestdf)

In [ ]:
pd.DataFrame(regpredict).to_csv('reg.csv')

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
tree=DecisionTreeRegressor()

In [ ]:
treemodel=tree.fit(X,y)

In [ ]:
treemodel.score(X,y)

In [ ]:
treepredict=treemodel.predict(housetestdf)

In [ ]:
pd.DataFrame(treepredict).to_csv('tree.csv')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
RF=RandomForestRegressor(n_estimators=3000)

In [ ]:
RFmodel=RF.fit(X,y)

In [ ]:
RFmodel.score(X,y)

In [ ]:
Rfpredict=RFmodel.predict(housetestdf)

In [ ]:
pd.DataFrame(Rfpredict).to_csv('RF.csv')

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbm=GradientBoostingRegressor(n_estimators=2000)

In [ ]:
gbmmodel=gbm.fit(X,y)

In [ ]:
gbmmodel.score(X,y)

In [ ]:
gbmpredict=gbmmodel.predict(housetestdf)

In [ ]:
pd.DataFrame(gbmpredict).to_csv('gbm.csv')